In [18]:
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import preprocessing
import math
import qgrid
import scipy as sc
from scipy import stats
from matplotlib.ticker import FuncFormatter

%matplotlib inline

In [8]:
PATHDATA = 'audiosData.csv'
data = pd.read_csv(PATHDATA)
data = data.values #Convertimos en un  numpy array
X = data[:,0:-2]
Y = data[:,-1]

In [23]:
def main(X,Y,hidden = (28,)):
    Folds = 15

    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    Y=  np.reshape(Y,(np.size(Y,0),1))
    groups = data[:,-2:-1]
    group_kfold = GroupKFold(n_splits=Folds)
    group_kfold.get_n_splits(X, Y, groups)
    j = 0
    for train, test in group_kfold.split(X, Y, groups):
        Xtrain = X[train,:]
        Ytrain = Y[train,:]
        Xtest = X[test,:]
        Ytest = Y[test,:]

        #Normalizamos los datos
        media = np.mean(Xtrain,axis=0)
        desvia = np.std(Xtrain,axis=0)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        mlp = MLPClassifier(activation='tanh', hidden_layer_sizes=hidden, max_iter=500 )
        mlp.fit(Xtrain,Ytrain)
        Ytrain_pred = mlp.predict(Xtrain)
        #Use para el modelo para hacer predicciones sobre el conjunto Xtest
        Yest = mlp.predict(Xtest)

        #Mida el error MAPE para cada una de las dos salidas
        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
        EficienciaVal[j] = np.mean(Yest == Ytest)
        j += 1
        
    mean = round(np.mean(EficienciaTrain),5)
    std = round(np.std(EficienciaTrain),5)
    
    meanVal = round(np.mean(EficienciaVal),5)
    stdVal = round(np.std(EficienciaVal),5)
    
    return mean,std,meanVal,stdVal



In [24]:
mean,std,meanVal,stdVal = main(X,Y,hidden= (28,))


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization h

NameError: name 'mean1' is not defined

In [26]:
print(mean, std, meanVal, stdVal)

0.25001 1e-05 0.25187 0.00171
